In [5]:
import numpy as np
import plotly.graph_objects as go
from deap import base, creator, tools, algorithms
import random

# Menghasilkan data dummy
np.random.seed(0)
data = np.random.rand(100)  # 100 data point
data = data.reshape(-1, 1)

# Normalisasi data
data_min = np.min(data)
data_max = np.max(data)
data_scaled = (data - data_min) / (data_max - data_min)

# Fungsi untuk membuat dataset
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Parameter LSTM
time_step = 10
X, Y = create_dataset(data_scaled, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Fungsi aktivasi sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Fungsi aktivasi tanh
def tanh(x):
    return np.tanh(x)

# Fungsi untuk inisialisasi bobot
def initialize_weights(input_dim, units):
    Wf = np.random.randn(input_dim, units) * 0.1  # Bobot untuk forget gate
    Wi = np.random.randn(input_dim, units) * 0.1  # Bobot untuk input gate
    Wo = np.random.randn(input_dim, units) * 0.1  # Bobot untuk output gate
    Wc = np.random.randn(input_dim, units) * 0.1  # Bobot untuk cell state
    return Wf, Wi, Wo, Wc

# Fungsi LSTM manual
def lstm_manual(X, units, epochs=50, learning_rate=0.001):
    input_dim = X.shape[2]
    Wf, Wi, Wo, Wc = initialize_weights(input_dim, units)
    
    # Inisialisasi state
    h = np.zeros((X.shape[0], units))
    c = np.zeros((X.shape[0], units))
    
    for epoch in range(epochs):
        for t in range(X.shape[1]):
            x_t = X[:, t, :]
            f_t = sigmoid(np.dot(x_t, Wf))  # Forget gate
            i_t = sigmoid(np.dot(x_t, Wi))  # Input gate
            o_t = sigmoid(np.dot(x_t, Wo))  # Output gate
            c_tilde = tanh(np.dot(x_t, Wc))  # Cell state candidate
            
            c = f_t * c + i_t * c_tilde  # Update cell state
            h = o_t * tanh(c)  # Update hidden state
    return h

# Fungsi untuk menghitung Mean Squared Error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Fungsi evaluasi untuk algoritma genetika
def evaluate(individual):
    units = int(individual[0])
    predictions = lstm_manual(X, units, epochs=50)
    # Mengembalikan skala data
    predictions = predictions * (data_max - data_min) + data_min
    Y_actual = data[time_step + 1:len(data)].reshape(-1, 1)
    mse = mean_squared_error(Y_actual, predictions)
    return mse,

# Pengaturan algoritma genetika
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimasi MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("units", random.randint, 10, 100)  # Rentang jumlah unit LSTM
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.units, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=50, sigma=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Menjalankan algoritma genetika
population = toolbox.population(n=10)
ngen = 10
for gen in range(ngen):
    fits = list(map(toolbox.evaluate, population))
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit

    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # Mating dan mutasi
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    population[:] = offspring

# Menampilkan hasil terbaik
fits = list(map(toolbox.evaluate, population))
for fit, ind in zip(fits, population):
    ind.fitness.values = fit

best_ind = tools.selBest(population, 1)[0]
print(f'Best LSTM units: {int(best_ind[0])}, MSE: {best_ind.fitness.values[0]}')

# Melatih model dengan parameter terbaik
best_units = int(best_ind[0])
predictions = lstm_manual(X, best_units, epochs=50)

# Mengembalikan skala data
predictions = predictions * (data_max - data_min) + data_min
Y_actual = data[time_step + 1:len(data)].reshape(-1, 1)

# Menampilkan grafik perbandingan
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(Y_actual)), y=Y_actual.flatten(), mode='lines', name='Data Asli'))
fig.add_trace(go.Scatter(x=np.arange(len(predictions)), y=predictions.flatten(), mode='lines', name='Prediksi LSTM Manual'))
fig.update_layout(title='Perbandingan Data Asli dan Prediksi LSTM Manual',
                  xaxis_title='Waktu',
                  yaxis_title='Nilai',
                  legend=dict(x=0, y=1))
fig.show()

Best LSTM units: 39, MSE: 0.27839618453541237


In [1]:
import numpy as np
import plotly.graph_objects as go
from deap import base, creator, tools, algorithms
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Menghasilkan data dummy
np.random.seed(0)
data = np.random.rand(100)  # 100 data point
data = data.reshape(-1, 1)

# Normalisasi data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Fungsi untuk membuat dataset
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

# Parameter LSTM
time_step = 10
X, Y = create_dataset(data_scaled, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

# Membuat model LSTM menggunakan Keras
def create_lstm_model(units):
    model = Sequential()
    model.add(LSTM(units, input_shape=(X.shape[1], 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Fungsi untuk menghitung Mean Squared Error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Fungsi evaluasi untuk algoritma genetika
def evaluate(individual):
    units = int(individual[0])
    model = create_lstm_model(units)
    model.fit(X, Y, epochs=50, batch_size=32, verbose=0)
    predictions = model.predict(X)
    mse = mean_squared_error(Y, predictions)
    return mse,

# Pengaturan algoritma genetika menggunakan DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimasi MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("units", random.randint, 10, 100)  # Rentang jumlah unit LSTM
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.units, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=50, sigma=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Menjalankan algoritma genetika
population = toolbox.population(n=10)
ngen = 10
for gen in range(ngen):
    # Evaluasi setiap individu dalam populasi
    fits = list(map(toolbox.evaluate, population))
    for fit, ind in zip(fits, population):
        ind.fitness.values = fit

    # Seleksi individu terbaik untuk generasi berikutnya
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    # Mating dan mutasi
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    population[:] = offspring

# Menampilkan hasil terbaik
fits = list(map(toolbox.evaluate, population))
for fit, ind in zip(fits, population):
    ind.fitness.values = fit

best_ind = tools.selBest(population, 1)[0]
print(f'Best LSTM units: {int(best_ind[0])}, MSE: {best_ind.fitness.values[0]}')

# Melatih model dengan parameter terbaik
best_units = int(best_ind[0])
best_model = create_lstm_model(best_units)
best_model.fit(X, Y, epochs=50, batch_size=32, verbose=0)
predictions = best_model.predict(X)

# Mengembalikan skala data
predictions = scaler.inverse_transform(predictions)
Y_actual = scaler.inverse_transform(Y.reshape(-1, 1))

# Menampilkan grafik perbandingan
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(Y_actual)), y=Y_actual.flatten(), mode='lines', name='Data Asli'))
fig.add_trace(go.Scatter(x=np.arange(len(predictions)), y=predictions.flatten(), mode='lines', name='Prediksi LSTM'))
fig.update_layout(title='Perbandingan Data Asli dan Prediksi LSTM dengan Optimasi GA',
                  xaxis_title='Waktu',
                  yaxis_title='Nilai',
                  legend=dict(x=0, y=1))
fig.show()

AttributeError: module 'numpy' has no attribute 'typeDict'